#上場株式データ取得


参考サイト

https://datatechlog.com/how-to-retrieve-stock-price-and-other-info-using-yfinance/

https://algotrading101.com/learn/yfinance-guide/



■抽出手順

全3763社（code.csv）から3000時点以上の値を有する会社を抽出する

時間がかかるため、分割して実行する（それぞれ約15分かかる）

range(0,1000)    →    0 ~  999 → df_0.csv

range(1000,2000) → 1000 ~ 1999 → df_1.csv

range(2000,3000) → 2000 ~ 2999 → df_2.csv

range(3000,3763) → 3000 ~ 3762 → df_3.csv


■抽出結果

df_0 → コード：1301 ~ 3919 → 716社 * 2640時点

df_1 → コード：3941 ~ 6236 → 621社 * 2658時点

df_2 → コード：6245 ~ 7983 → 727社 * 2709時点

df_3 → コード：7984 ~ 9997 → 666社 * 2689時点

#ライブラリ読み込み

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
# ライブラリの読み込み
import yfinance as yf
import pandas as pd
import numpy as np

In [15]:
code = pd.read_csv('/content/drive/MyDrive/data/code/code.csv')

#範囲

In [16]:
range_start = 1000
range_end = 2000

#関数定義

In [17]:
def df_range(range_start, range_end):

  # 3000時点以上の時系列を有する株価時系列を抽出（i = 0 スタート）
  code_dict = {}
  j = 0
  i = range_start

  while True:
    df = yf.download(str(code.iloc[i,0]) + '.T', start='2000-01-01', end='2021-12-31')['Adj Close']
    print(i, code.iloc[i,0], code.iloc[i,1], df.shape)
    next_start = i + 1

    if len(df) > 3000:
      code_dict[j] = [code.iloc[i,0], code.iloc[i,1]]
      j += 1
      break
    i += 1

  for i in range(next_start, range_end):
    df_app = yf.download(str(code.iloc[i,0]) + '.T', start='2000-01-01', end='2021-12-31')['Adj Close']
    print(i, code.iloc[i,0], code.iloc[i,1], df_app.shape)
    if len(df_app) > 3000:
      df = pd.merge(df, df_app, how = 'inner', on = 'Date')
      code_dict[j] = [code.iloc[i,0], code.iloc[i,1]]
      j += 1

  list = []

  for i in range(len(code_dict)):
    list.append(code_dict[i][0])
  df.index.name = 'timestamp'
  df = df.set_axis(list, axis = 'columns')

  return df

# 実行

In [18]:
df = df_range(range_start, range_start + 100)

for i in range(1, 10):
  df = pd.merge(df, df_range(range_start + i * 100, range_start + (i+1)*100), how = 'inner', on = 'timestamp')

[*********************100%***********************]  1 of 1 completed
1000 3929 ソーシャルワイヤー (1492,)
[*********************100%***********************]  1 of 1 completed
1001 3930 はてな (1451,)
[*********************100%***********************]  1 of 1 completed
1002 3931 バリューゴルフ (1446,)
[*********************100%***********************]  1 of 1 completed
1003 3932 アカツキ (1436,)
[*********************100%***********************]  1 of 1 completed
1004 3933 チエル (1433,)
[*********************100%***********************]  1 of 1 completed
1005 3934 ベネフィットジャパン (1431,)
[*********************100%***********************]  1 of 1 completed
1006 3935 エディア (1416,)
[*********************100%***********************]  1 of 1 completed
1007 3936 グローバルウェイ (1413,)
[*********************100%***********************]  1 of 1 completed
1008 3937 Ｕｂｉｃｏｍホールディングス (1372,)
[*********************100%***********************]  1 of 1 completed
1009 3939 カナミックネットワーク (1313,)
[*********************100%*********************

In [19]:
df.to_csv("df_1.csv")

from google.colab import files
files.download('df_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>